In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_meta = pd.read_csv('../metadata/train.csv')

In [3]:
val_meta = pd.read_csv('../metadata/valid.csv')

In [4]:
base_path = '../real_vs_fake/real-vs-fake/'

In [5]:
image_list = []
for x in train_meta['path']:
    image_list.append(base_path + x)

In [6]:
val_image_list = []
for x in val_meta['path']:
    val_image_list.append(base_path + x)

In [7]:
len(image_list)

100000

In [8]:
def image_processing(batch_size):
    batch_processed = []
    for x in range(batch_size[0],batch_size[1]):
        img = Image.open(image_list[x])
        img = img.resize((224,224))
        image_array = np.array(img) / 255.0
        batch_processed.append(image_array)
    return batch_processed

In [ ]:

batches = [(0, 15000), (15001, 30000), (30001, 45000), (45001, 60000), (60001, 75000), (75001, 90000), (90001, 100000)]
X_train = []
for batch in batches:
    processed_batch = image_processing(batch)
    X_train.append(processed_batch)
    


In [ ]:
len(x)

In [ ]:
len(X_train)

In [ ]:
X_valid = []
for x in range(len(val_image_list)):
    image = np.array(Image.open(val_image_list[x]))
    X_valid.append(image)

In [ ]:
y_valid = val_meta['label']

In [ ]:
y_train = train_meta['label']

In [ ]:
def create_cnn(filters=32, kernel_size=(3, 3), pool_size=(2, 2), dense_units=128):
    model = Sequential()
    model.add(Convolution2D(filters, kernel_size, input_shape=(256, 256, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
param_grid = {
    'filters': [32, 64],
    'kernel_size': [(3, 3), (5, 5)],
    'pool_size': [(2, 2), (3, 3)],
    'dense_units': [128, 256]}

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
cnn_model = KerasClassifier(build_fn=create_cnn, epochs=5, batch_size=64, verbose=0)
grid = GridSearchCV(estimator=cnn_model, param_grid=param_grid, scoring='accuracy', cv=3)

In [ ]:
grid_result = grid.fit(X_train, y_train, validation_data=(X_valid, y_valid))

In [ ]:
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)